# S02: 特徴量エンジニアリング

このノートブックでは以下の処理を行います：
1. 全HTMLファイルをパースして中間ファイル（CSV）を生成
2. 特徴量の計算
3. 学習用データセットの作成

**実行環境**: Sagemaker

---
## 1. 環境設定

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルート
PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# データパス
HTML_DIR = PROJECT_ROOT / 'data' / 'raceHTML'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
FEATURES_DIR = PROJECT_ROOT / 'data' / 'features'

# ディレクトリ作成
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

print(f"HTML_DIR: {HTML_DIR.resolve()}")
print(f"PROCESSED_DIR: {PROCESSED_DIR.resolve()}")
print(f"FEATURES_DIR: {FEATURES_DIR.resolve()}")

In [ ]:
# モジュールインポート
from scraper.parser import parse_multiple_html_full, classify_race_level
from features.calculator import calculate_horse_features, build_feature_dataset

print("✅ モジュール読み込み完了")

---
## 2. 中間ファイル生成

全HTMLをパースして以下のCSVを生成：
- `races.csv`: レース情報
- `results.csv`: 出走馬・レース結果

In [ ]:
# 既存の中間ファイルを確認
races_csv = PROCESSED_DIR / 'races.csv'
results_csv = PROCESSED_DIR / 'results.csv'

if races_csv.exists() and results_csv.exists():
    print(f"✅ 中間ファイルが既に存在します")
    print(f"   races.csv: {races_csv.stat().st_size / 1024 / 1024:.2f} MB")
    print(f"   results.csv: {results_csv.stat().st_size / 1024 / 1024:.2f} MB")
    print("\n再生成する場合は次のセルを実行してください")
else:
    print("中間ファイルが見つかりません。次のセルでHTMLをパースします。")

In [ ]:
# HTMLパース実行（初回または再生成時）
def progress_callback(processed, total):
    print(f"\r進捗: {processed}/{total} ({processed/total*100:.1f}%)", end='')

print("📊 HTMLパースを開始します...")
print(f"対象ディレクトリ: {HTML_DIR}")

# 全年のHTMLをパース
race_list, horse_list = parse_multiple_html_full(
    HTML_DIR,
    years=None,  # 全年
    progress_callback=progress_callback
)

print(f"\n\n✅ パース完了")
print(f"   レース数: {len(race_list):,}")
print(f"   出走馬データ数: {len(horse_list):,}")

In [ ]:
# DataFrameに変換して保存
races_df = pd.DataFrame(race_list)
results_df = pd.DataFrame(horse_list)

# CSVに保存
races_df.to_csv(races_csv, index=False, encoding='utf-8')
results_df.to_csv(results_csv, index=False, encoding='utf-8')

print(f"✅ 中間ファイル保存完了")
print(f"   {races_csv}: {races_csv.stat().st_size / 1024 / 1024:.2f} MB")
print(f"   {results_csv}: {results_csv.stat().st_size / 1024 / 1024:.2f} MB")

---
## 3. データ確認

In [ ]:
# 中間ファイル読み込み
races_df = pd.read_csv(races_csv)
results_df = pd.read_csv(results_csv)

print("=== レースデータ ===")
print(f"件数: {len(races_df):,}")
print(f"\nカラム:")
print(races_df.columns.tolist())

print("\n=== 出走馬データ ===")
print(f"件数: {len(results_df):,}")
print(f"\nカラム:")
print(results_df.columns.tolist())

In [ ]:
# レースデータのサンプル
print("=== レースデータ サンプル ===")
races_df.head(3)

In [ ]:
# 出走馬データのサンプル
print("=== 出走馬データ サンプル ===")
results_df.head(5)

In [ ]:
# 年別レース数
if 'year' in races_df.columns:
    print("=== 年別レース数 ===")
    year_counts = races_df.groupby('year').size()
    print(year_counts.to_string())

In [ ]:
# レースレベル分布
if 'race_level' in races_df.columns:
    print("=== レースレベル分布 ===")
    level_counts = races_df['race_level'].value_counts().sort_index()
    print(level_counts.to_string())

In [ ]:
# 欠損値確認
print("=== 出走馬データ 欠損値 ===")
missing = results_df.isnull().sum()
missing_pct = (missing / len(results_df) * 100).round(2)
missing_df = pd.DataFrame({'欠損数': missing, '欠損率(%)': missing_pct})
print(missing_df[missing_df['欠損数'] > 0].sort_values('欠損率(%)', ascending=False).head(20))

---
## 4. 抽出された項目一覧

### レース情報 (races.csv)

| カラム | 説明 |
|--------|------|
| race_id | レースID（12桁） |
| venue_code | 競馬場コード |
| venue_name | 競馬場名 |
| kaisai | 開催回 |
| day | 開催日 |
| race_num | レース番号 |
| race_name | レース名 |
| date | 日付 |
| year | 年 |
| month | 月 |
| surface | 芝/ダート |
| distance | 距離 |
| direction | 右/左回り |
| weather | 天候 |
| track_condition | 馬場状態 |
| start_time | 発走時刻 |
| horse_count | 出走頭数 |
| first_prize | 1着賞金 |
| race_level | レースレベル |
| level_score | レベルスコア |

### 出走馬データ (results.csv)

| カラム | 説明 |
|--------|------|
| race_id | レースID |
| horse_id | 馬ID |
| horse_name | 馬名 |
| finish_position | 着順 |
| is_finished | 完走フラグ |
| dnf_reason | 失格理由 |
| gate_number | 枠番 |
| horse_number | 馬番 |
| sex | 性別 |
| age | 年齢 |
| impost | 斤量 |
| jockey_id | 騎手ID |
| jockey_name | 騎手名 |
| trainer_id | 調教師ID |
| trainer_name | 調教師名 |
| trainer_region | 調教師所属（東/西） |
| owner_id | 馬主ID |
| owner_name | 馬主名 |
| time_str | タイム（文字列） |
| time_seconds | タイム（秒） |
| margin | 着差 |
| passing_order | 通過順 |
| first_corner | 1コーナー順位 |
| last_corner | 最終コーナー順位 |
| last_3f | 上がり3F |
| odds | 単勝オッズ |
| popularity | 人気 |
| horse_weight | 馬体重 |
| weight_change | 馬体重増減 |
| prize_money | 獲得賞金 |
| race_level | レースレベル |
| level_score | レベルスコア |
| race_date | レース日 |
| venue_code | 競馬場コード |
| venue_name | 競馬場名 |
| distance | 距離 |
| surface | 芝/ダート |
| track_condition | 馬場状態 |

---
## 5. 次のステップ

中間ファイルが生成されました。以降は `results.csv` を使って：

1. **特徴量計算**: `calculate_horse_features()` で馬ごとの過去成績を集計
2. **モデル学習**: `S03_model_training.ipynb` で予測モデルを学習
3. **予測**: `L02_prediction.ipynb` でローカル予測